In [71]:
import pandas as pd
import glob
from tqdm import tqdm
import json

### Prepare data for eval! 

In [62]:
df = pd.read_pickle('/home/amey/projects/multilingual-long-context-LLMs/prompts/cot_synthetic/zs_ar_2.pkl')
df.head(1)

,id,input_lang,prompt,target,predictions_/home/models/deepseek-llm-7b-chat_vllm_temperature_0.0,predictions_/home/models/Meta-Llama-3.1-8B-Instruct_vllm_temperature_0.0,aggregated_target
0,83_ar,ar,"Read the provided text carefully. First, trans...",حمام,"{\n ""reasoning"": ""The text mentions that Yo...",**Step 1: Identify and extract only the releva...,"[bathroom, स्नानघर, baño, Badezimmer, phòng tắ..."


In [ ]:
print(df.iloc)

In [57]:
df_groundtruth = pd.read_csv('/home/amey/projects/multilingual-long-context-LLMs/data/babilong/multilingual_bAbi_groundtruth.csv')
targets = [eval(x) for x in df_groundtruth.groupby("question_type").sample(n=10, random_state=42)['target'].values.tolist()]
df['aggregated_target'] = targets
df

,id,input_lang,prompt,target,predictions_/home/models/Mistral-7B-Instruct-v0.2_vllm_temperature_0.0,predictions_/home/models/Qwen2-7B-Instruct_vllm_temperature_0.0,predictions_/home/models/Meta-Llama-3.1-8B-Instruct_vllm_temperature_0.0,aggregated_target
0,83_zh,zh,"Read the provided text carefully. First, trans...",浴室,"{\n ""reasoning"": ""1. Identify and extract ...","{\n ""reasoning"": ""Step 1: Identify and extr...",**Step 1: Identify and extract only the releva...,"[bathroom, स्नानघर, baño, Badezimmer, phòng tắ..."
1,53_zh,zh,"Read the provided text carefully. First, trans...",花园,Step 1: Identify and extract only the relevan...,"{\n ""reasoning"": ""Step 1: Identify and extr...",**Step 1: Identify and extract only the releva...,"[garden, बगीचा, jardín, Garten, vườn, 花园, حديقة]"
2,70_zh,zh,"Read the provided text carefully. First, trans...",办公室,Step 1: Identify and extract only the relevan...,"{\n ""reasoning"": ""Step 1: Identify and extr...",**Step 1: Identify and extract only the releva...,"[office, कार्यालय, oficina, Büro, văn phòng, 办..."
3,45_zh,zh,"Read the provided text carefully. First, trans...",办公室,Step 1: Identify and extract only the relevan...,"{\n ""reasoning"": ""Step 1: Identify and extr...",**Step 1: Identify and extract only the releva...,"[office, कार्यालय, oficina, Büro, văn phòng, 办..."
4,44_zh,zh,"Read the provided text carefully. First, trans...",办公室,"{\n ""reasoning"": [\n ""Step 1: Ident...","{\n ""reasoning"": ""Step 1: Identify and extr...",**Step 1: Identify and extract only the releva...,"[office, कार्यालय, oficina, Büro, văn phòng, 办..."
...,...,...,...,...,...,...,...,...
95,867_zh,zh,"Read the provided text carefully. First, trans...",是的,Step 1: Identify any statements in the contex...,"{\n ""reasoning"": ""Step 1: Identify any stat...",**Step 1: Identify any statements that include...,"[yes, हाँ, Sí, Ja, Đúng, 是的, نعم]"
96,889_zh,zh,"Read the provided text carefully. First, trans...",不,Step 1: Identify any statements that include ...,"{\n ""reasoning"": ""Step 1: Identify any stat...",**Step 1: Identify any statements that include...,"[no, नहीं, No, NEIN, KHÔNG, 不, لا]"
97,842_zh,zh,"Read the provided text carefully. First, trans...",不,Step 1: Identify any statements in the contex...,"{\n ""reasoning"": ""Step 1: Identify any stat...",**Step 1: Identify any statements that include...,"[no, नहीं, No, NEIN, KHÔNG, 不, لا]"
98,828_zh,zh,"Read the provided text carefully. First, trans...",是的,Step 1: Identify any statements that include ...,"{\n ""reasoning"": ""Step 1: Identify any stat...",**Step 1: Identify any statements that include...,"[yes, हाँ, Sí, Ja, Đúng, 是的, نعم]"


In [42]:
import os

def find_pkl_files(directory):
    pkl_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pkl'):
                full_path = os.path.join(root, file)
                pkl_files.append(full_path)
                # print(full_path)  # Print each found path
    return pkl_files

# Example usage
directory_path = 'prompts/'
pkl_file_paths = find_pkl_files(directory_path)

In [43]:
df_groundtruth = pd.read_csv('/home/amey/projects/multilingual-long-context-LLMs/data/babilong/multilingual_bAbi_groundtruth.csv')

In [46]:
'predictions_/home/models/Meta-Llama-3.1-8B-Instruct_vllm_temperature_0.0'.split('/')[-1].split('_')[0]

'Meta-Llama-3.1-8B-Instruct'

In [54]:
eval_files = []
prediction_cols = ['predictions_/home/models/deepseek-llm-7b-chat_vllm_temperature_0.0', 'predictions_/home/models/Meta-Llama-3.1-8B-Instruct_vllm_temperature_0.0']

for filename in pkl_file_paths:
    df = pd.read_pickle(filename)
    print(filename)
    
    experiment = filename.replace('.pkl','')
    baseline = experiment.split('/')[1].split('_')[0]
    distractor_type = experiment.split('/')[1].split('_')[1]
    prompt_type = experiment.split('/')[-1].split('_')[0]
    context_lang = experiment.split('/')[-1].split('_')[1]
    prompt_size = experiment.split('/')[-1].split('_')[2]
    print(baseline, distractor_type, prompt_type, context_lang, prompt_size)
    
    """    
    targets = [eval(x) for x in df_groundtruth.groupby("question_type").sample(n=10, random_state=42)['target'].values.tolist()]
    df['aggregated_target'] = targets
    df.to_pickle(filename)
    """
    for col in prediction_cols:
        if col in df.columns and int(prompt_size) <= 12 :
            model_name = col.split('/')[-1].split('_')[0]
            eval_files.append({
                'input_file': filename,
                'baseline': baseline,
                'distractor_type': distractor_type,
                'prompt_type': prompt_type,
                'context_lang': context_lang,
                'prompt_size': prompt_size,
                'prediction_col': col,
                'model_name': ''
            })

    print('-'*50)
    # break


prompts/cot_mlqa/zs_ar_320.pkl
cot mlqa zs ar 320
--------------------------------------------------
prompts/cot_mlqa/zs_zh_20.pkl
cot mlqa zs zh 20
--------------------------------------------------
prompts/cot_mlqa/zs_hi_2.pkl
cot mlqa zs hi 2
--------------------------------------------------
prompts/cot_mlqa/zs_en_80.pkl
cot mlqa zs en 80
--------------------------------------------------
prompts/cot_mlqa/zs_vi_1000.pkl
cot mlqa zs vi 1000
--------------------------------------------------
prompts/cot_mlqa/zs_de_80.pkl
cot mlqa zs de 80
--------------------------------------------------
prompts/cot_mlqa/zs_vi_20.pkl
cot mlqa zs vi 20
--------------------------------------------------
prompts/cot_mlqa/zs_ar_500.pkl
cot mlqa zs ar 500
--------------------------------------------------
prompts/cot_mlqa/zs_es_80.pkl
cot mlqa zs es 80
--------------------------------------------------
prompts/cot_mlqa/zs_en_1000.pkl
cot mlqa zs en 1000
--------------------------------------------------


In [55]:
print(f"Total files: {len(pkl_file_paths)}")
print(f"Eligible files: {len(eval_files)}")
print(f"Missing files: {len(pkl_file_paths) - len(eval_files)}")

Total files: 756
Eligible files: 350
Missing files: 406


In [58]:
eval_files[0]

{'input_file': 'prompts/cot_mlqa/zs_hi_2.pkl',
 'baseline': 'cot',
 'distractor_type': 'mlqa',
 'prompt_type': 'zs',
 'context_lang': 'hi',
 'prompt_size': '2',
 'prediction_col': 'predictions_/home/models/deepseek-llm-7b-chat_vllm_temperature_0.0',
 'model_name': ''}

In [69]:
output_dir = 'results_acl_2025'
command_template = lambda input_csv, savefile, prediction_col : f"""
python compute_accuracy.py \
--input_csv {input_csv} \
--savefile {savefile} \
--prediction_col {prediction_col} \
--reference_col aggregated_target \
--similarity_threshold 0.95 \
; 
""".strip()

for eval_file in eval_files:
    command = command_template(
        input_csv=eval_file['input_file'],
        savefile=f"results_acl_2025/{eval_file['prompt_type']}_{eval_file['baseline']}_{eval_file['distractor_type']}_{eval_file['context_lang']}_{eval_file['prompt_size']}.csv",
        prediction_col=eval_file['prediction_col']
    )
    print(command)
    print("-"*50)

python compute_accuracy.py --input_csv prompts/cot_mlqa/zs_hi_2.pkl --savefile results_acl_2025/zs_cot_mlqa_hi_2.csv --prediction_col predictions_/home/models/deepseek-llm-7b-chat_vllm_temperature_0.0 --reference_col aggregated_target --similarity_threshold 0.95 ;
--------------------------------------------------
python compute_accuracy.py --input_csv prompts/cot_mlqa/zs_vi_8.pkl --savefile results_acl_2025/zs_cot_mlqa_vi_8.csv --prediction_col predictions_/home/models/deepseek-llm-7b-chat_vllm_temperature_0.0 --reference_col aggregated_target --similarity_threshold 0.95 ;
--------------------------------------------------
python compute_accuracy.py --input_csv prompts/cot_mlqa/zs_vi_10.pkl --savefile results_acl_2025/zs_cot_mlqa_vi_10.csv --prediction_col predictions_/home/models/deepseek-llm-7b-chat_vllm_temperature_0.0 --reference_col aggregated_target --similarity_threshold 0.95 ;
--------------------------------------------------
python compute_accuracy.py --input_csv prompts/cot

In [67]:
results = pd.read_csv('/home/amey/projects/multilingual-long-context-LLMs/results_acl_2025/zs_translation_mlqa_hi_2.csv')
results.head(1)

,id,input_lang,prompt,target,predictions_/home/models/deepseek-llm-7b-chat_vllm_temperature_0.0,predictions_/home/models/Meta-Llama-3.1-8B-Instruct_vllm_temperature_0.0,aggregated_target,prediction_cleaned,accuracy_exact_match,accuracy
0,83_hi,hi,"Read the provided text carefully. First, trans...",स्नानघर,"{\n ""ANSWER"": ""भरतवंश""\n}","{\n ""ANSWER"": ""बाथरूम""\n}","['bathroom', 'स्नानघर', 'baño', 'badezimmer', ...",बाथरूम,0,0


In [70]:
# print(results.iloc[0]['prompt'])
print(results.iloc[0]['predictions_/home/models/Meta-Llama-3.1-8B-Instruct_vllm_temperature_0.0'])
print(results.iloc[0]['aggregated_target'])
print(results.iloc[0]['accuracy'])

{
    "ANSWER": "बाथरूम"
}
['bathroom', 'स्नानघर', 'baño', 'badezimmer', 'phòng tắm', '浴室', 'حمام']
0


### Analyse data post eval

In [24]:
import pandas as pd
import glob
from tqdm import tqdm

results_dir = 'results_acl_2025/*.csv'
df_results = pd.DataFrame(columns=['prompt_type', 'baseline', 'distractor_type', 'context_lang', 'prompt_size', 'exact_accuracy'])

for filename in tqdm(glob.glob(results_dir)):
    df = pd.read_csv(filename)
    exp = filename.split('/')[-1].split('.')[0]
    mean_acc = float(df['accuracy'].mean())

    results_dict = {
        'prompt_type': exp.split('_')[0],
        'baseline': exp.split('_')[1],
        'distractor_type': exp.split('_')[2],
        'context_lang': exp.split('_')[3],
        'prompt_size': exp.split('_')[4],
        'exact_accuracy': mean_acc
    }

    try:
        df_results = pd.concat([df_results, pd.DataFrame([results_dict])], ignore_index=True)
    except Exception as e:
        print(f"Error: {e}")
        df_results = pd.DataFrame([results_dict])

# Optionally, save the results
df_results.to_csv('aggregated_results.csv', index=False)


  0%|          | 0/210 [00:00<?, ?it/s]/tmp/ipykernel_2623757/1591037854.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, pd.DataFrame([results_dict])], ignore_index=True)
100%|██████████| 210/210 [00:03<00:00, 55.94it/s]


In [25]:
df_results['context_lang'].value_counts()

context_lang
es    30
en    30
ar    30
de    30
vi    30
hi    30
zh    30
Name: count, dtype: int64

In [54]:
# Display the new structured dataframe
df_results['prompt_size'] = df_results['prompt_size'].apply(lambda x: int(x))
df_results.head()

,prompt_type,baseline,distractor_type,context_lang,prompt_size,exact_accuracy
0,zs,translation,mlqa,es,12,0.43
1,zs,translation,synthetic,es,8,0.38
2,zs,translation,synthetic,en,12,0.54
3,zs,translation,synthetic,ar,2,0.35
4,zs,cot,mlqa,de,2,0.32


In [55]:
df_view = df_results[df_results['distractor_type'] == 'random']
df_view.head()

,prompt_type,baseline,distractor_type,context_lang,prompt_size,exact_accuracy
8,zs,translation,random,hi,2,0.29
10,zs,cot,random,hi,10,0.82
20,zs,translation,random,en,10,0.60
29,zs,translation,random,ar,8,0.38
32,zs,translation,random,zh,8,0.39


In [56]:
df_view['baseline'].value_counts()

baseline
translation    35
cot            35
Name: count, dtype: int64

In [57]:
import pandas as pd


# Assuming df_results is your original dataframe
df_pivot = df_view.pivot(index=['prompt_type', 'baseline', 'distractor_type', 'context_lang'], 
                            columns='prompt_size', 
                            values='exact_accuracy').reset_index()

# Rename columns for clarity
df_pivot.columns.name = None  # Remove hierarchical column name
df_pivot = df_pivot.rename_axis(None, axis=1)  # Ensure a clean DataFrame


df_pivot


,prompt_type,baseline,distractor_type,context_lang,2,4,8,10,12
0,zs,cot,random,ar,0.81,0.82,0.82,0.82,0.78
1,zs,cot,random,de,0.79,0.81,0.79,0.79,0.80
2,zs,cot,random,en,0.81,0.83,0.82,0.78,0.84
3,zs,cot,random,es,0.83,0.81,0.83,0.82,0.80
4,zs,cot,random,hi,0.82,0.83,0.80,0.82,0.81
5,zs,cot,random,vi,0.77,0.81,0.80,0.82,0.80
6,zs,cot,random,zh,0.75,0.77,0.77,0.78,0.81
7,zs,translation,random,ar,0.47,0.45,0.38,0.43,0.38
8,zs,translation,random,de,0.51,0.49,0.56,0.55,0.53
9,zs,translation,random,en,0.56,0.58,0.60,0.60,0.52


In [59]:
# Sort the DataFrame based on 'prompt_size' to group same values together
# df_sorted = df_pivot.sort_values(by=['prompt_size'])
df_sorted = df_pivot.sort_values(by=['baseline'])

df_sorted.head(100)


,prompt_type,baseline,distractor_type,context_lang,2,4,8,10,12
0,zs,cot,random,ar,0.81,0.82,0.82,0.82,0.78
1,zs,cot,random,de,0.79,0.81,0.79,0.79,0.80
2,zs,cot,random,en,0.81,0.83,0.82,0.78,0.84
3,zs,cot,random,es,0.83,0.81,0.83,0.82,0.80
4,zs,cot,random,hi,0.82,0.83,0.80,0.82,0.81
5,zs,cot,random,vi,0.77,0.81,0.80,0.82,0.80
6,zs,cot,random,zh,0.75,0.77,0.77,0.78,0.81
7,zs,translation,random,ar,0.47,0.45,0.38,0.43,0.38
8,zs,translation,random,de,0.51,0.49,0.56,0.55,0.53
9,zs,translation,random,en,0.56,0.58,0.60,0.60,0.52


In [51]:
df_sorted.to_csv('result_table_acl_2025/random.csv')